# 📊 Feature Engineering for Stock Clustering

**Goal**: Transform raw stock prices into meaningful risk indicators.

**Why?** Clustering algorithms need numeric features that capture different aspects of risk:
- **Volatility**: How much the price swings
- **Returns**: Profitability patterns
- **Technical Indicators**: Market sentiment signals
- **Liquidity**: How easy to trade
- **Risk-adjusted performance**: Return vs risk tradeoff

In [15]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')

from features import (
    calculate_returns,
    calculate_volatility_features,
    calculate_risk_metrics,
    calculate_technical_indicators,
    calculate_liquidity_features,
    calculate_momentum_features,
    calculate_drawdown,
    aggregate_stock_features
)

## 1️⃣ Load Cleaned Data

In [16]:
df = pd.read_csv('../Data/Processed/cleaned_nse.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69754 entries, 0 to 69753
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            69754 non-null  object 
 1   Stock_code      69754 non-null  object 
 2   Name            69754 non-null  object 
 3   12m Low         69754 non-null  float64
 4   12m High        69754 non-null  float64
 5   Day Low         69754 non-null  float64
 6   Day High        69754 non-null  float64
 7   Day Price       69754 non-null  float64
 8   Previous        69754 non-null  float64
 9   Change          69754 non-null  float64
 10  %Change         69754 non-null  float64
 11  Volume          69754 non-null  float64
 12  Adjusted Price  69754 non-null  float64
 13  Sector          69754 non-null  object 
 14  Month           69754 non-null  int64  
 15  Year            69754 non-null  int64  
dtypes: float64(10), int64(2), object(4)
memory usage: 8.5+ MB


In [17]:
df['Stock_code'].head()

0    ABSA
1    ABSA
2    ABSA
3    ABSA
4    ABSA
Name: Stock_code, dtype: object

## 2️⃣ Feature Engineering Pipeline

We'll apply 7 transformations to create ~25 features per stock:

### A) Returns (Profitability)
Daily returns show how much profit/loss each day

In [19]:
# STEP 1
print("Step 1/7: Calculating returns...")
print(f"Before - Index: {df.index.name}, Stock_code in columns? {'Stock_code' in df.columns}")

# Handle the case where Stock_code is both index and column
if df.index.name == 'Stock_code' and 'Stock_code' in df.columns:
    # If it's both, just drop the index (don't try to add it again)
    df = df.reset_index(drop=True)
    print("Dropped index (Stock_code already in columns)")
elif df.index.name == 'Stock_code':
    # If it's only index, reset it to column
    df = df.reset_index()
    print("Reset Stock_code from index to column")

# Now set Stock_code as index
df = df.set_index('Stock_code')
print(f"After set_index - Index: {df.index.name}")

# Apply calculations
df = df.groupby(level=0, group_keys=False).apply(
    calculate_returns, include_groups=False
)
print(f"After groupby - Index: {df.index.name}, Stock_code in columns? {'Stock_code' in df.columns}")

# Reset index safely
if 'Stock_code' in df.columns:
    df = df.reset_index(drop=True)
    print("Dropped index (Stock_code already in columns)")
else:
    df = df.reset_index()
    print("Reset index to column")

print(f"After reset_index - Stock_code in columns? {'Stock_code' in df.columns}")
print(f"✅ Added: daily_return, log_return")
print(f"Final columns: {df.columns.tolist()}")

Step 1/7: Calculating returns...
Before - Index: None, Stock_code in columns? True
After set_index - Index: Stock_code
After groupby - Index: Stock_code, Stock_code in columns? True
Dropped index (Stock_code already in columns)
After reset_index - Stock_code in columns? True
✅ Added: daily_return, log_return
Final columns: ['Date', 'Name', '12m Low', '12m High', 'Day Low', 'Day High', 'Day Price', 'Previous', 'Change', '%Change', 'Volume', 'Adjusted Price', 'Sector', 'Month', 'Year', 'daily_return', 'log_return', 'Stock_code']


### B) Volatility (Price Swings)
Standard deviation of returns = how unpredictable the stock is

In [20]:
# STEP 2
print("Step 2/7: Calculating volatility...")
df = df.set_index('Stock_code')
df = df.groupby(level=0, group_keys=False).apply(
    calculate_volatility_features, include_groups=False
)
df = df.reset_index()
print(f"✅ Added: volatility_7d, volatility_14d, volatility_30d")

Step 2/7: Calculating volatility...
✅ Added: volatility_7d, volatility_14d, volatility_30d


### C) Advanced Risk Metrics
- **Downside deviation**: Only measures bad volatility (losses)
- **Value at Risk (VaR)**: "5% chance of losing this much or more"

In [21]:
# STEP 3: Advanced Risk Metrics
print("Step 3/7: Calculating risk metrics...")
df = df.set_index('Stock_code')
df = df.groupby(level=0, group_keys=False).apply(
    calculate_risk_metrics, include_groups=False
)
df = df.reset_index()
print(f"✅ Added: downside_deviation_30d, var_95")


Step 3/7: Calculating risk metrics...
✅ Added: downside_deviation_30d, var_95


### D) Technical Indicators
- **RSI** (0-100): <30 = oversold, >70 = overbought
- **Bollinger Bands**: Volatility envelope around price
- **MACD**: Trend momentum indicator

In [22]:
# STEP 4: Technical Indicators
print("Step 4/7: Calculating technical indicators...")
df = df.set_index('Stock_code')
df = df.groupby(level=0, group_keys=False).apply(
    calculate_technical_indicators, include_groups=False
)
df = df.reset_index()
print(f" Added: rsi, bb_width, bb_position, macd, macd_signal")
print(f"Columns now: {df.columns.tolist()}")  

Step 4/7: Calculating technical indicators...
 Added: rsi, bb_width, bb_position, macd, macd_signal
Columns now: ['Stock_code', 'Date', 'Name', '12m Low', '12m High', 'Day Low', 'Day High', 'Day Price', 'Previous', 'Change', '%Change', 'Volume', 'Adjusted Price', 'Sector', 'Month', 'Year', 'daily_return', 'log_return', 'volatility_7d', 'volatility_14d', 'volatility_30d', 'downside_deviation_30d', 'var_95', 'rsi', 'bb_width', 'bb_position', 'macd', 'macd_signal']


### E) Liquidity Features
Can you buy/sell easily? High volume = liquid, low volume = illiquid (risky)

In [23]:
# STEP 5: Liquidity Features
print("Step 5/7: Calculating liquidity features...")
df = df.set_index('Stock_code')
df = df.groupby(level=0, group_keys=False).apply(
    calculate_liquidity_features, include_groups=False
)
df = df.reset_index()
print(f" Added: avg_volume, volume_volatility, volume_trend, amihud_illiquidity")
print(f"Stock_code in columns? {'Stock_code' in df.columns}")  # Verification

Step 5/7: Calculating liquidity features...
 Added: avg_volume, volume_volatility, volume_trend, amihud_illiquidity
Stock_code in columns? True


### F) Momentum and Trends
Is the stock going up/down/sideways? Comparing current price to moving averages

In [24]:
# STEP 6: Momentum Features
print("Step 6/7: Calculating momentum features...")
df = df.set_index('Stock_code')
df = df.groupby(level=0, group_keys=False).apply(
    calculate_momentum_features, include_groups=False
)
df = df.reset_index()
print(f" Added: momentum_7d, momentum_30d, momentum_90d, ma_7, ma_30, ma_50, price_to_ma30, price_to_ma50")
print(f"Stock_code in columns? {'Stock_code' in df.columns}")  # Verification

Step 6/7: Calculating momentum features...
 Added: momentum_7d, momentum_30d, momentum_90d, ma_7, ma_30, ma_50, price_to_ma30, price_to_ma50
Stock_code in columns? True


### G) Drawdown (Crash Risk)
**Max Drawdown**: Largest peak-to-trough decline. Shows worst-case scenario.

In [28]:
# STEP 7: Drawdown Metrics
print("Step 7/7: Calculating drawdown metrics...")
df = df.set_index('Stock_code')
df = df.groupby(level=0, group_keys=False).apply(
    calculate_drawdown, include_groups=False  # This uses the imported function
)
df = df.reset_index()
print(f"✅ Added: current_drawdown, max_drawdown, days_from_peak")
print(f"Stock_code in columns? {'Stock_code' in df.columns}")

print(f"\nFeature engineering complete!")
print(f"Final DataFrame shape: {df.shape}")
print(f"Final columns: {df.columns.tolist()}")

Step 7/7: Calculating drawdown metrics...


KeyError: 'close'

## 3️⃣ Aggregate to Stock Level

**Problem**: We have ~1000 rows per stock (one per day)

**Solution**: Take **averages/medians** to get ONE row per stock

**Key aggregated features**:
- **Volatility**: mean, max
- **Returns**: mean, std, skew, kurtosis
- **Sharpe Ratio**: Return per unit of risk (CRUCIAL!)
- **Technical**: RSI mean, Bollinger width, MACD volatility
- **Liquidity**: volume, trading frequency, illiquidity
- **Risk**: max drawdown, VaR, downside deviation

In [27]:
print("Aggregating features to stock level...")

features_list = []
for stock_code, group in df.groupby('Stock_code'):
    stock_features = aggregate_stock_features(group)
    if stock_features is not None:
        features_list.append(stock_features)

df_features = pd.DataFrame(features_list)
print(f"\n✅ Created {len(df_features)} stock profiles with {len(df_features.columns)} features")
df_features.head()

Aggregating features to stock level...


KeyError: 'max_drawdown'

## 4️⃣ Inspect Key Features

In [ ]:
print("Feature Statistics:\n")
print(df_features[[
    'volatility_mean', 'sharpe_ratio', 'max_drawdown', 
    'trading_frequency', 'rsi_mean', 'downside_deviation'
]].describe().round(4))

## 5️⃣ Save Features

In [ ]:
output_path = '../Data/Processed/nse_features.csv'
df_features.to_csv(output_path, index=False)
print(f"✅ Saved features to {output_path}")

---

## 📚 Summary

**What we did**:
1. ✅ Calculated returns and volatility (basic risk)
2. ✅ Added advanced risk metrics (downside dev, VaR)
3. ✅ Computed technical indicators (RSI, Bollinger, MACD)
4. ✅ Measured liquidity (volume, illiquidity)
5. ✅ Tracked momentum and trends (MAs, price ratios)
6. ✅ Analyzed drawdowns (max loss)
7. ✅ Aggregated ~1000 daily rows → 1 stock profile

**Key insight**: Clustering works MUCH better with diverse features that capture different risk dimensions.

**Next**: Use these features for K-Means clustering! 🎯